# Velogames solver: Giro d'Italia 2023

A script to calculate the optimal team that could have been chosen for a given race in [Velogames fantasy cycling](https://www.velogames.com/)

This Julia script uses the [Gumbo](https://github.com/JuliaWeb/Gumbo.jl) and [Cascadia](https://github.com/Algocircle/Cascadia.jl) libraries to scrape rider data, and the [JuMP](https://jump.dev/JuMP.jl/stable/) optimisation library / [HiGHS](https://highs.dev/) solver to construct and solve a mixed-integer program (MIP) described below

In Velogames fantasy cycling, you must select a team of 9 riders, each with a specific cost based on their expected performance, spending no more than 100 points. 

Each rider is classed as either an All-Rounder, a Climber, a Sprinter or is Unclassed. A team must contain 2 All-Rounders, 2 Climbers, 1 Sprinter and 3 Unclassed riders. The 9th selection can be from any of these categories.

At the end of the race, each rider will have accumulated a score based on their performance, and the aim is to pick a team with the highest combined score at the end of the race.

The optimisation problem can be stated as:

$maximise \sum_{j=1}^{n} x_j y_j$

$s.t.$ 

$\sum_{j=1}^{n} x_j=9$

$\sum_{j=1}^{n} x_j z_j \leq 100$

$\sum_{j=1}^{n} x_j a_j \geq 2$

$\sum_{j=1}^{n} x_j c_j \geq 2$

$\sum_{j=1}^{n} x_j s_j \geq 1$

$\sum_{j=1}^{n} x_j u_j \geq 3$

where $j=1...n$ is the set of all riders

$x_j\in[0,1]$ is a binary decision variable denoting if rider $j$ is chosen (1 for chosen, 0 for not chosen)

$z_j\in Z^+$ and $y_j\in Z^+$ are the cost and score parameters of rider $j$ respectively

$a_j\in[0,1]$, $c_j\in[0,1]$, $s_j\in[0,1]$ and $u_j\in[0,1]$ are binary parameters denoting if rider $j$ is an All-Rounder, Climber, Sprinter or Unclassed respectively, with the further parameter constraint that $a_i+c_i+s_i+u_i=1$ $\forall i=1...n$ (i.e. each rider is allocated to one and only one of the 4 categories) and by implication $\sum_{j=1}^{n} a_j+\sum_{j=1}^{n} c_j+\sum_{j=1}^{n} s_j+\sum_{j=1}^{n} u_j=n$ (i.e. the sum of the number of riders in each category is equal to the total number of riders)

## Load libraries

In [9]:
# include all files in src directory
source_files = readdir("src")
for file in source_files
    include("src/$file")
end

In [12]:
rider_df = getvgriders("https://www.velogames.com/italy/2023/riders.php")



rider_df

Row,,rider,team,class,cost,selected,points,riderkey,All Rounder,Climber,Sprinter,Unclassed,value
,String,String,String,String,Int64,String,Int64,String,Bool,Bool,Bool,Bool,Float64
1,,Remco Evenepoel,Soudal - Quick Step,All Rounder,26,68.7%,402,ceeeeelmnooprv,true,false,false,false,15.4615
2,,Primož Roglič,Jumbo-Visma,All Rounder,24,53.7%,289,giilmooprr,true,false,false,false,12.0417
3,,Geraint Thomas,INEOS Grenadiers,All Rounder,18,11.3%,148,aaeghimnorstt,true,false,false,false,8.22222
4,,Tao Geoghegan Hart,INEOS Grenadiers,Climber,16,25.2%,272,aaaeeggghhnoortt,false,true,false,false,17.0
5,,João Almeida,UAE Team Emirates,All Rounder,16,38.6%,285,aadeijlmoo,true,false,false,false,17.8125
6,,Aleksandr Vlasov,BORA - hansgrohe,All Rounder,14,12.9%,186,aaadekllnorssvv,true,false,false,false,13.2857
7,,Mads Pedersen,Trek - Segafredo,Sprinter,14,39.0%,256,addeeemnprss,false,false,true,false,18.2857
8,,Jack Haig,Bahrain - Victorious,Climber,12,10.6%,29,aacghijk,false,true,false,false,2.41667
9,,Thymen Arensman,INEOS Grenadiers,All Rounder,12,4.7%,20,aaeehmmnnnrsty,true,false,false,false,1.66667


In [13]:
model = Model(HiGHS.Optimizer)
@variable(model, x[rider_df.rider], Bin)
@objective(model, Max, rider_df.points' * x) # maximise the total score
@constraint(model, rider_df.cost' * x <= 100) # cost must be <= 100
@constraint(model, sum(x) == 9) # exactly 9 riders must be chosen
@constraint(model, rider_df[!, "All Rounder"]' * x >= 2) # at least 2 must be all rounders
@constraint(model, rider_df[!, "Sprinter"]' * x >= 1) # at least 1 must be a sprinter
@constraint(model, rider_df[!, "Climber"]' * x >= 2) # at least 2 must be climbers
@constraint(model, rider_df[!, "Unclassed"]' * x >= 3) # at least 3 must be unclassed
optimize!(model)

Running HiGHS 1.5.1 [date: 1970-01-01, git hash: 93f1876e4]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
6 rows, 176 cols, 528 nonzeros
6 rows, 97 cols, 253 nonzeros
6 rows, 95 cols, 248 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   6 rows
   95 cols (77 binary, 18 integer, 0 implied int., 0 continuous)
   248 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   8447            -inf                 inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   2630.5          1995              31.85%        0      0      0         9     0.0s
 C       0       0         0   0.00%   2627            2027              29.60%       20      2      4        11     0.0s

Solving re

In [14]:
# total score
objective_value(model)

2627.0

In [15]:
# total cost
rider_df.cost.*value.(x).data |> sum

100.0

In [16]:
# selected riders
rider_df[!,:chosen] = value.(x).data .|>  !iszero
filter(:chosen => ==(true), rider_df)

Row,,rider,team,class,cost,selected,points,riderkey,All Rounder,Climber,Sprinter,Unclassed,value,chosen
,String,String,String,String,Int64,String,Int64,String,Bool,Bool,Bool,Bool,Float64,Bool
1,,Remco Evenepoel,Soudal - Quick Step,All Rounder,26,68.7%,402,ceeeeelmnooprv,true,false,false,false,15.4615,true
2,,Tao Geoghegan Hart,INEOS Grenadiers,Climber,16,25.2%,272,aaaeeggghhnoortt,false,true,false,false,17.0,true
3,,João Almeida,UAE Team Emirates,All Rounder,16,38.6%,285,aadeijlmoo,true,false,false,false,17.8125,true
4,,Michael Matthews,Team Jayco AlUla,Sprinter,10,10.4%,358,aaceehhilmmsttw,false,false,true,false,35.8,true
5,,Koen Bouwman,Jumbo-Visma,Climber,8,3.0%,146,abekmnnoouw,false,true,false,false,18.25,true
6,,Aurélien Paret-Peintre,AG2R Citroën Team,Unclassed,6,8.3%,300,aaeeeeiilnnpprrrttu,false,false,false,true,50.0,true
7,,Vincenzo Albanese,EOLO-Kometa,Sprinter,6,5.8%,336,aabceeeilnnnosvz,false,false,true,false,56.0,true
8,,Andreas Leknessund,Team DSM,Unclassed,6,5.6%,255,aaddeeeklnnnrsssu,false,false,false,true,42.5,true
9,,Toms Skujiņš,Trek - Segafredo,Unclassed,6,1.1%,273,ijkmosstu,false,false,false,true,45.5,true
